Importação Arquivo para o Google Colab

In [ ]:
!cp /content/drive/MyDrive/Engenharia-De-Software/Big-Data/imdb-reviews-pt-br.csv /content

Instalacao pyspark

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e1179d5459412c768bec25fb779ae787e4210699900b4867f0e2de74d19be7b6
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


Importar a biblioteca e criar uma seção Spark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Criar DataFrame com o Spark

In [ ]:
imdb_df = spark.read.csv('imdb-reviews-pt-br.csv',
                         header=True,
                         quote="\"",
                         escape="\"",
                         encoding="UTF-8")

Mostrar o resultado

In [ ]:
imdb_df.show()

+---+--------------------+--------------------+---------+
| id|             text_en|             text_pt|sentiment|
+---+--------------------+--------------------+---------+
|  1|Once again Mr. Co...|Mais uma vez, o S...|      neg|
|  2|This is an exampl...|Este é um exemplo...|      neg|
|  3|First of all I ha...|Primeiro de tudo ...|      neg|
|  4|Not even the Beat...|Nem mesmo os Beat...|      neg|
|  5|Brass pictures mo...|Filmes de fotos d...|      neg|
|  6|A funny thing hap...|Uma coisa engraça...|      neg|
|  7|This German horro...|Este filme de ter...|      neg|
|  8|Being a long-time...|Sendo um fã de lo...|      neg|
|  9|"Tokyo Eyes" tell...|"Tokyo Eyes" fala...|      neg|
| 10|Wealthy horse ran...|Fazendeiros ricos...|      neg|
| 11|Cage plays a drun...|Cage interpreta u...|      neg|
| 12|First of all, I w...|Primeiro de tudo,...|      neg|
| 13|So tell me - what...|Então me diga - q...|      neg|
| 14|A big disappointm...|Uma grande decepç...|      neg|
| 15|This film

Devo criar um código utilizando funções MAP e REDUCE para exibir a soma dos ID's de comentários classificados como 'Neg' ou Negativo

In [ ]:
# Criação da função MAP para mapear o sententiment como chave e o id como valor inteiro
def funcao_map(row):
    sentimento = row['sentiment']
    valor_id = int(row['id'])
    return (sentimento, valor_id)

# Criação da função REDUCE para somar os IDs
def funcao_reduce(x, y):
    return x + y

# Aplicação das funções MAP e REDUCE
rdd_mapeado = imdb_df.rdd.map(funcao_map)
rdd_reduzido = rdd_mapeado.reduceByKey(funcao_reduce)

# Coletar resultados
resultado = rdd_reduzido.collect()

# Exibir a soma dos IDs classificados como 'negativo'
for sentimento, id_total in resultado:
    if sentimento == 'neg':
        print("Soma dos IDs classificados como 'negativo':", id_total)

Já aqui devo criar um código com funções MAP e REDUCE para exibir a diferença entre a soma das palavras dos textos em português e inglês classificados como "neg"

In [ ]:
from pyspark.sql.functions import col

# Criando a sessão Spark
spark = SparkSession.builder.getOrCreate()

# Função MAP para mapear o sentimento como chave e uma tupla com a soma das palavras de cada texto
def contagem_palavras_map(row):
    sentimento = row['sentiment']
    texto_ingles = len(row['text_en'].split())
    texto_portugues = len(row['text_pt'].split())
    return (sentimento, (texto_ingles, texto_portugues))

# Função REDUCE para somar o número total de palavras dos textos em inglês e português por sentimento
def soma_palavras_red(x, y):
    return (x[0] + y[0], x[1] + y[1])

# Aplicando o MAP e REDUCE no DataFrame
rdd_map_red = imdb_df.rdd.map(contagem_palavras_map).reduceByKey(soma_palavras_red)

# Coletando os resultados
resultados = rdd_map_red.collect()

# Selecionando os dados relacionados aos textos negativos
contagem_neg = dict(resultados)['neg']

# Calculando a diferença entre as somas de palavras dos textos em português e inglês classificados como negativos
diferenca = contagem_neg[1] - contagem_neg[0]

print("Diferença entre a soma das palavras em português e inglês dos textos classificados como 'neg'é", diferenca,)

Diferença entre a soma das palavras em português e inglês dos textos classificados como 'neg'é 54949
